In [17]:
import tensorflow as tf
import pandas  as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
# Here I've proposed a Neural Network that uses the Bidirectional LSTM cells for the processing of our feature vectors .

# HYPERPARAMS
vocab_size = 12000
maxlen = 120
embedding_dim = 20

df = pd.read_csv("train.csv")
sentence , label = df.loc[:,"Statement"].values , df.loc[:,"Label"].values

label = np.where(label==True,1,0)
label

array([0, 1, 1, ..., 1, 0, 0])

In [19]:
# Prepairing the tokenizer 
tokenizer = Tokenizer(num_words=vocab_size,oov_token = "<OOV>")
tokenizer.fit_on_texts(sentence)
sequences = tokenizer.texts_to_sequences(sentence)

print(tokenizer.word_index)
# Padding the data
pad_sequence = pad_sequences(sequences,maxlen,padding="pre")


x_train,x_test,y_train,y_test = train_test_split(pad_sequence,label,test_size=0.2,stratify=label)

# Prepairing the Model

model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,input_length=maxlen),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32,activation="relu"),
    tf.keras.layers.Dense(64,activation="relu"),
    tf.keras.layers.Dense(1,activation="sigmoid")
])

model.compile(loss="binary_crossentropy",optimizer=tf.keras.optimizers.RMSprop(lr=0.01,momentum=0.9,decay=0.4),metrics=["accuracy"])

history = model.fit(x_train,y_train,validation_data=(x_test,y_test),epochs=20,verbose=1)

#model.save("Fake_news.h5")

{'<OOV>': 1, 'the': 2, 'in': 3, 'of': 4, 'to': 5, 'a': 6, 'and': 7, 'says': 8, 'for': 9, 'that': 10, 'is': 11, 'on': 12, 'has': 13, 'have': 14, 'percent': 15, 'are': 16, 'than': 17, 'was': 18, 'more': 19, 'we': 20, 'by': 21, 'state': 22, 'from': 23, 'it': 24, 'with': 25, '000': 26, 'as': 27, 'obama': 28, 'tax': 29, 'not': 30, 'our': 31, 'year': 32, 'health': 33, 'years': 34, 'president': 35, 'people': 36, 'he': 37, 'states': 38, 'at': 39, 'would': 40, 'million': 41, 'an': 42, 'be': 43, 'care': 44, 'his': 45, 'one': 46, 'i': 47, 'you': 48, 'their': 49, 'were': 50, 'this': 51, 'jobs': 52, 'over': 53, 'will': 54, 'new': 55, 'u': 56, 's': 57, 'been': 58, 'they': 59, 'or': 60, '1': 61, 'who': 62, 'about': 63, 'when': 64, 'billion': 65, 'all': 66, 'bill': 67, 'texas': 68, 'only': 69, 'out': 70, 'every': 71, 'said': 72, 'taxes': 73, 'under': 74, 'there': 75, 'federal': 76, 'no': 77, 'voted': 78, 'barack': 79, 'since': 80, 'up': 81, 'government': 82, 'budget': 83, 'had': 84, 'if': 85, 'now': 8

In [23]:
sentence = ["America is in very dangerous state right now"]
sequence = tokenizer.texts_to_sequences(sentence)
model.predict(sequence)

array([[0.5626597]], dtype=float32)

In [0]:
df_test = pd.read_csv("test.csv")
test,test_label = df_test.loc[:,"Statement"].values , df_test.loc[:,"Label"].values
test_label = np.where(test_label==True,1,0)
scores = model.evaluate(test,test_label)
scores